In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import json
import nltk
import matplotlib.pyplot as plt
import counter

In [2]:
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

def clean_text(text):
    # Ensure consistent spacing after titles like "Item 1A."
    text = re.sub(r'(Item\s*\d+[A-Za-z]*)\.', r'\1. ', text)
    # Insert missing spaces between concatenated words like "FactorsThe"
    text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)
    # Clean up spacing after punctuation
    text = re.sub(r'(?<=[.!?])(?=\S)', r' ', text)
    return text.strip()

def split_sentences_with_nltk(text):
    # Initial cleaning
    text = clean_text(text)

    # Use nltk's sentence tokenizer
    sentences = nltk.sent_tokenize(text)

    # Post-process to remove redundant or empty sentences
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    return sentences

[nltk_data] Downloading package punkt to /Users/ajaymdn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
def converttotext(url):
    headers = {'User-Agent':'Sample Company Name AdminContact@<sample company domain>.com','Accept-Encoding':'gzip, deflate','Host':'www.sec.gov'}

    r = requests.get(url, headers=headers)

    raw_10k = r.text

    doc_start_pattern = re.compile(r'<DOCUMENT>')
    doc_end_pattern = re.compile(r'</DOCUMENT>')
    type_pattern = re.compile(r'<TYPE>[^\n]+')

    doc_start_is = [x.end() for x in doc_start_pattern.finditer(raw_10k)]
    doc_end_is = [x.start() for x in doc_end_pattern.finditer(raw_10k)]

    doc_types = [x[len('<TYPE>'):] for x in type_pattern.findall(raw_10k)]

    document = {}
    
    for doc_type, doc_start, doc_end in zip(doc_types, doc_start_is, doc_end_is):
        if doc_type == '10-K':
            document[doc_type] = raw_10k[doc_start:doc_end]

    regex = re.compile(r'(>Item(\s|&#160;|&nbsp;)(1A|1B|7A|7|8)\.{0,1})|(ITEM\s(1A|1B|7A|7|8))')

    matches = regex.finditer(document['10-K'])

    test_df = pd.DataFrame([(x.group(), x.start(), x.end()) for x in matches])

    test_df.columns = ['item', 'start', 'end']
    test_df['item'] = test_df.item.str.lower()
    test_df.head()
    test_df.replace('&#160;',' ',regex=True,inplace=True)
    test_df.replace('&nbsp;',' ',regex=True,inplace=True)
    test_df.replace(' ','',regex=True,inplace=True)
    test_df.replace('\.','',regex=True,inplace=True)
    test_df.replace('>','',regex=True,inplace=True)

    pos_dat = test_df.sort_values('start', ascending=True).drop_duplicates(subset=['item'], keep='last')
    pos_dat.set_index('item', inplace=True)

    item_1a_raw = document['10-K'][pos_dat['start'].loc['item1a']:pos_dat['start'].loc['item1b']]

    item_1a_content = BeautifulSoup(item_1a_raw, 'lxml')

    item_1a_text = item_1a_content.get_text()

    sentences = split_sentences_with_nltk(item_1a_text)
    
    return sentences


In [4]:
def load_esg_dictionaries(json_file_path="words.json"):
    with open(json_file_path, 'r') as file:
        esg_dictionaries = json.load(file)
    return esg_dictionaries

In [5]:
def preprocess_text(text):
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text

In [ ]:
esg_dictionaries = load_esg_dictionaries()



In [30]:
def analyze_esg(text, esg_dictionaries):
    text = preprocess_text(text)
    word_counts = Counter(text.split())
    
    # Initialize scores for Environmental, Social, and Governance
    environmental_score = 0
    social_score = 0
    governance_score = 0
    
    # Calculate the scores based on the keyword matches
    for keyword in esg_dictionaries['Environmental']:
        environmental_score += word_counts.get(keyword, 0)
    for keyword in esg_dictionaries['Social']:
        social_score += word_counts.get(keyword, 0)
    for keyword in esg_dictionaries['Governance']:
        governance_score += word_counts.get(keyword, 0)
    
    # Normalize the scores by the total word count
    total_words = sum(word_counts.values())
    if total_words > 0:
        environmental_score /= total_words
        social_score /= total_words
        governance_score /= total_words
    
    # Return the individual ESG scores (normalized)
    return environmental_score, social_score, governance_score

In [33]:
esg_dictionaries = load_esg_dictionaries()
text = converttotext("https://www.sec.gov/Archives/edgar/data/1018724/000101872420000004/0001018724-20-000004.txt")
environmental, social, governance = analyze_esg(" ".join(text), esg_dictionaries)

print(f"Environmental: {environmental:.4f}")
print(f"Social: {social:.4f}")
print(f"Governance: {governance:.4f}")

Environmental: 0.0106
Social: 0.0144
Governance: 0.0120
